# Calculate the glass transition temperature, $T_g$, of an atomistic amorphous system using simulation tools at nanoHUB.org

In [1]:
# System setup
import sys, os, tempfile

#sys.path.append('/apps/share64/debian7/ipynb_tools')
sys.path.append('/storage/davidelbert/persistent/NanoHUB_API_Example/NanoHUB_remote')

# Run this calculation in a temporary directory
workdir = tempfile.mkdtemp(dir=os.getcwd())
#%cd $workdir

In [2]:
from NanoHUB_remote import \
    authenticate, get_driver, launch_tool, get_results, extract_results

from mysecrets import auth_data

# Authenticate; use headers in all subsequent steps
headers = authenticate(auth_data)

In [3]:
# The short name of the nanoHUB tool to run; this is the final stanza of
# the tool URL (e.g. https://nanohub.org/tools/complam)
tool_name = 'dftqe'

# Input values; keys are the labels of the inputs in the GUI
tool_inputs = {
}

In [4]:
# Generate the XML driver to run the tool with our inputs
driver_json = get_driver(tool_name, tool_inputs, headers)

In [5]:
# Start the simulation
session_id = launch_tool(driver_json, headers)
print session_id

1336122


In [6]:
run_results = get_results(session_id, headers)

In [7]:
print run_results

<?xml version="1.0"?>
<run>
    <tool>
        <id>dftqe</id>
        <name>DFT calculations with Quantum ESPRESSO</name>
        <version>
            <identifier>2.9.2</identifier>
            <application>
                <revision>111</revision>
                <modified>2016-03-24 19:32:11 -0400 (Thu, 24 Mar 2016)</modified>
                <installed>2016-03-24 19:34:31 EDT</installed>
                <directory id="top">/apps/dftqe/r111</directory>
                <directory id="tool">/apps/dftqe/r111/rappture</directory>
            </application>
            <rappture>
                <version>1.7</version>
                <revision>6697</revision>
            </rappture>
        </version>
        <title>nanoMaterials Simulation Toolkit - Quantum Espresso DFT calculations</title>
        <about>
      Use the four panels on the left to set up the simulation.
        -Input model: specify the initial atomic model.
        -Energy expression: level of theory, approximations and

In [ ]:
# View the initial structure built by PolymerModeler
import nglview as nv
import subprocess as sp

tmp_pdb = 'temp.pdb'
polymod_pdb = 'polymod.pdb'
with open(tmp_pdb, 'w') as f:
    f.write(get_output_pdb('Built structures, unwrapped'))
sp.call(['babel', tmp_pdb, polymod_pdb])  # add CONECT records
v = nv.show_structure_file(polymod_pdb)
v.representations = [{'type': 'ball+stick', 'params': {}}]
v.parameters = {'backgroundColor': 'black'}
v

In [ ]:
# Relax and thermalize the initial structure built by PolymerModeler
import hublib.use
%use lammps-15May15

nodes = 2
walltime = '1:00:00'
datafile = 'polymer_relax.data'
with open(datafile, 'w') as f:
    f.write(task.get_output_value('LAMMPS data file'))
infile = 'lammps_relax.in'
with open(infile, 'w') as f:
    f.write(task.get_output_value('LAMMPS input file'))

!submit -n $nodes -w $walltime -i $datafile lammps-15May15-parallel -in $infile

In [ ]:
# Cool and compress the structure with LAMMPS
infile = 'lammps_cool.in'
T_steps = 10000
lammps_input = """
read_restart      restart.lammps

neighbor          2.0 bin
thermo_style      custom temp density vol etotal ke pe ebond eangle edihed eimp evdwl ecoul elong press pxx pyy pzz pxy pxz pyz lx ly lz
thermo            100
thermo_modify     flush yes line multi format float %16.6g

timestep          4
neigh_modify      every 1 delay 5
kspace_style      pppm 1e-4
run_style         respa 3 2 2 bond 1 pair 2 kspace 3
velocity          all create 800.0 1214556 mom yes rot yes dist gaussian

# Loop
variable          i loop 26
label             loopa
reset_timestep    0
variable          T equal 800+($i-1)*(-20)

fix               1 all npt temp $T $T 100.0 iso 1.0 1.0 1000.
dump              2 all custom 500 strcool_$TK.dump id type x y z
run               {0}
write_data        str_cool$TK.data
unfix             1
undump            2
next              i
jump              {1} loopa
""".format(T_steps, infile)

with open(infile, 'w' ) as f:
    f.write(lammps_input)
walltime = '12:00:00'

!submit -n $nodes -w $walltime -i restart.lammps lammps-15May15-parallel -in $infile

In [ ]:
# Extract the density and temperature data from LAMMPS output; find Tg
from Tgcalc import getVDT, getVDTdata, getTg_BL
import matplotlib.pyplot as plt
%matplotlib notebook

getVDT('log.lammps', T_steps)
data_file = 'log.rec_VDT'

# Plot density vs. T
T, rho, err = getVDTdata(data_file)
plt.style.use('ggplot')
plt.plot(T, rho, 'b-')
plt.xlabel('T (K)')
plt.ylabel('Density (g/cc)')
plt.show()

# Calculate Tg
print 'Tg from bilinear fit:', getTg_BL(data_file)